#Shruti Jawale - Decision Tree

##Import libraries

In [1]:
import math
import sklearn
import csv
import sys
import pandas as pd
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import resample
from sklearn.metrics import recall_score, precision_score, accuracy_score, confusion_matrix

##Set up the dataset

###One Hot Encoding

In [7]:
global amazondata, features, upsampleddata
inputDataset = input("enter the full path from current directory to the input dataset file:\n")
amazondata = pd.read_csv(inputDataset)

categories = pd.get_dummies(amazondata[['category']])
features = ['stars', 'reviews', 'price', 'listPrice', 'boughtInLastMonth'] + list(categories.columns)

amazondata = amazondata.drop(amazondata.columns[[0,1,6]], axis=1)
amazondata = amazondata[['isBestSeller', 'stars', 'reviews', 'price', 'listPrice', 'boughtInLastMonth']]
amazondata = pd.concat([amazondata, categories], axis=1)

amazondata.head()

enter the full path from current directory to the input dataset file:
processedFile.csv


,isBestSeller,stars,reviews,price,listPrice,boughtInLastMonth,category_Abrasive & Finishing Products,category_Accessories & Supplies,category_Additive Manufacturing Products,category_Arts & Crafts Supplies,...,category_Women's Accessories,category_Women's Clothing,category_Women's Handbags,category_Women's Jewelry,category_Women's Shoes,category_Women's Watches,"category_Xbox 360 Games, Consoles & Accessories","category_Xbox One Games, Consoles & Accessories","category_Xbox Series X & S Consoles, Games & Accessories",category_eBook Readers & Accessories
0,False,4.5,0,139.99,0.00,2000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,False,4.5,0,169.99,209.99,1000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,False,4.6,0,365.49,429.99,300,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,4.6,0,291.59,354.37,400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,4.5,0,174.99,309.99,400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Upsampling the dataset

In [12]:
datafalse = amazondata[amazondata.isBestSeller == False]
datatrue = amazondata[amazondata.isBestSeller == True]
datatrueupsampled = resample(datatrue, replace = True, n_samples = 85200)

upsampleddata = pd.concat([datafalse, datatrueupsampled])
print(upsampleddata['isBestSeller'].value_counts())

False    1417817
True       85200
Name: isBestSeller, dtype: int64


##Training the data

In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(upsampleddata.get(features), upsampleddata['isBestSeller'], test_size=0.3)

classifier = DecisionTreeClassifier(random_state=0, max_depth=130)
classifier.fit(xtrain, ytrain)

DecisionTreeClassifier(max_depth=130, random_state=0)

##Testing the data

In [19]:
ypred = classifier.predict(xtest)
print("depth of tree:", classifier.get_depth(), "\nnumber of leaves:", classifier.get_n_leaves())

depth of tree: 123 
number of leaves: 17291


##Check results

In [15]:
accuracytest = accuracy_score(ytest, ypred)
recalltest = recall_score(ytest, ypred)
precisiontest = precision_score(ytest, ypred)
print("accuracy on test =", accuracytest)
print("recall on test =", recalltest)
print("precision on test =", precisiontest)

conf = confusion_matrix(ytest, ypred)
print("confusionmatrix on test=\n", conf)

accuracy on test = 0.9892233457028944
recall on test = 0.9751165040007034
precision on test = 0.8552699228791774
confusionmatrix on test=
 [[561458   5630]
 [   849  33270]]
